# **모델 사용하기**

## **1.환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(langchain)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [2]:
# 경로 : /content/drive/MyDrive/langchain/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/langchain/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### 2) 라이브러리 로딩

In [3]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (3) OpenAI API Key 확인

In [4]:
def load_api_key(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

path = '/content/drive/MyDrive/langchain/'

# API 키 로드 및 환경변수 설정
openai.api_key = load_api_key(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2.랭체인 모델 사용하기1**
* 입력

### (1) 랭체인으로 LLM 모델 연결

In [6]:
# langeuage_models
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model = "gpt-3.5-turbo")

### (2) HumanMessage

In [7]:
result = chat([HumanMessage(content="안녕하세요!")])
print(result.content)

안녕하세요! 도와드릴게 있나요?


### (3) SystemMessage

In [8]:
sys_role = '당신은 애국심을 가지고 있는 건전한 대한민국 국민입니다.'
question = "독도는 어느나라 땅이야?"

result = chat([HumanMessage(content = question), SystemMessage(content = sys_role)])
print(result.content)

독도는 대한민국의 땅입니다. 일본이 주장하는 영유권 요구는 역사적으로 부당하며, 국제사회에서도 대한민국의 주권이 인정되고 있습니다. 함께 대한민국의 영토를 지키고 지킬 수 있도록 노력해야 합니다.


### (4) PromptTemplate

* 프롬프트 템플릿 설정

In [9]:
from langchain import PromptTemplate

prompt = PromptTemplate(template = "{nation}의 인구수는?", input_variables = ["nation"])

In [10]:
print(prompt.format(nation = "한국"))
print(prompt.format(nation = "영국"))

한국의 인구수는?
영국의 인구수는?


* 언어 모델과 결합

In [11]:
result = chat([HumanMessage(content=prompt.format(nation = "한국"))])
print(result.content)

2021년 9월 기준 한국의 인구수는 약 51,829,023명입니다.


### 😀실습

* 다음 제공되는 국가 리스트(nations)를 이용해서
* 해당 국가의 인구수를 숫자로 뽑아 리스트에 담아 봅시다.

In [14]:
nations = ['한국','중국','일본','인도','미국','영국']
prompt = PromptTemplate(template = "{nation}의 인구수를 알려줘. 답변은 다른 내용은 전혀 필요 없고 숫자 만 알려줘.",
                        input_variables = ["nation"])

In [16]:
population = []
for nation in nations:
    result = chat([HumanMessage(content=prompt.format(nation = nation))])
    print(result.content)
    population.append(result.content)

population

5152만명 (2021년 7월 기준)
14억명.
1억2641만명 (2021년)
13억 7천만명.
미국의 인구는 약 331,449,281 명이다.
영국의 인구수는 약 67,886,011명입니다.


['5152만명 (2021년 7월 기준)',
 '14억명.',
 '1억2641만명 (2021년)',
 '13억 7천만명.',
 '미국의 인구는 약 331,449,281 명이다.',
 '영국의 인구수는 약 67,886,011명입니다.']

## **3.랭체인 모델 사용하기2**
* 출력

### (1) 랭체인으로 LLM 모델 연결

In [17]:
# langeuage_models
chat = ChatOpenAI(model = "gpt-3.5-turbo")

### (2) Output Parser

In [18]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [19]:
# 출력파서 선언
output_parser = CommaSeparatedListOutputParser()

# 사용
result = chat([HumanMessage(content = "트랜스포머 기반 언어모델 3개 알려줘."),
               HumanMessage(content = output_parser.get_format_instructions())])

# 결과 출력1
print(result.content)

# 결과 출력2
output = output_parser.parse(result.content)
print(output)

GPT-3, BERT, T5
['GPT-3', 'BERT', 'T5']


### 😀실습

* 충청도의 대표 관광지 5개를 리스트 형태로 출력해 봅시다.

In [22]:
# 출력파서 선언
output_parser = CommaSeparatedListOutputParser()

# 사용
result = chat([HumanMessage(content = "충북 단양의 대표 관광지 5개를 알려줘. 한글로 답변해."),
               HumanMessage(content = output_parser.get_format_instructions())])

# 결과 출력1
print(result.content)

# 결과 출력2
output = output_parser.parse(result.content)
print(output)

삼봉산, 단양 석재공원, 단양 얼음골, 두물머리 자연휴양림, 단양 스카이워크
['삼봉산', '단양 석재공원', '단양 얼음골', '두물머리 자연휴양림', '단양 스카이워크']


### (3) Streaming
* 답변을 생성하는 대로 화면에 순차적으로 출력

In [ ]:
# 기존 방식대로
chat1 = ChatOpenAI(model="gpt-3.5-turbo")

sys_role = '당신은 세계 최고의 요리사이자, 요리 선생님입니다. 당신은 항상 친절하고 정확하게 요리법을 가르칩니다.'
question = "한국식 후라이드치킨 만드는 방법 알려줘?"

result = chat1([HumanMessage(content=question), SystemMessage(content = sys_role)])
print(result.content)

In [24]:
# 스트리밍 방식으로 출력
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat2 = ChatOpenAI(model="gpt-3.5-turbo",
                   streaming=True, callbacks=[StreamingStdOutCallbackHandler()])  # 스트리밍 설정

sys_role = '당신은 세계 최고의 요리사이자, 요리 선생님입니다. 당신은 항상 친절하고 정확하게 요리법을 가르칩니다.'
question = "한국식 후라이드치킨 만드는 방법 알려줘?"
result = chat2([HumanMessage(content=question), SystemMessage(content = sys_role)])

한국식 후라이드치킨을 만드는 방법은 다음과 같습니다:

재료:
- 닭 다리살 또는 닭 날개
- 밀가루
- 계란
- 빵가루
- 식용유

조리법:
1. 먼저 닭 다리살이나 닭 날개를 소금과 후추로 조금씩 조미한 후 냉장고에서 30분 정도 숙성시킵니다.
2. 밀가루, 계란, 빵가루를 각각 별도의 그릇에 넣습니다. 계란은 풀어주어야 합니다.
3. 닭 다리살이나 닭 날개를 밀가루에 묻힌 후 계란에 푹 젖게 합니다.
4. 마지막으로 빵가루에 묻혀 깊은 팬에 식용유를 두르고 중불에서 앞뒤로 5분씩 튀겨줍니다.
5. 후라이드치킨이 골고루 노릇해질 때까지 튀겨낸 후 종이타올에 놓아 기름을 제거해줍니다.
6. 후라이드치킨을 맛있게 먹기 위해 양념소스나 간장소스 등과 함께 내놓습니다.

이렇게하면 집에서도 부드럽고 바삭한 한국식 후라이드치킨을 만들 수 있습니다. 맛있게 즐기세요!